In [1]:
#  word level CTC  tensorflow version 1.3

# this module requires the following :

#  a dataset csv file ( have a look at dataset csv generator)
#  a dictionary.txt file ( have a look at dictionary folder)



# to do :   
#   , get rid of fixed batch size , instead use variable batch size  


# perform following tests :

# 1. variable batch sizes test
# 2. GPU test





# please change the following according to your system
# hyperparameters in this notebook

# change paths in this notebook to that corresponding to your system
# please note number of audio examples in dataset must be even 

# peace.




In [2]:
import tensorflow as tf
from __future__ import print_function
from tensorflow.contrib import rnn
import numpy as np
import scipy.io.wavfile as wav
from python_speech_features import mfcc
from tensorflow.contrib.data import Dataset, Iterator
import time


/home/saurabh/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
def audio_to_mfcc(fileurl):
    rate, sig = wav.read(fileurl)
    mfcc_feat = mfcc(sig,rate, nfilt=55,numcep=15)
#d_mfcc_feat = delta(mfcc_feat, 2)
#fbank_feat = logfbank(sig,rate)

    return mfcc_feat

In [4]:
def normalized(a, axis=-1, order=2):
    l2 = np.atleast_1d(np.linalg.norm(a, order, axis))
    l2[l2==0] = 1
    return a / np.expand_dims(l2, axis)


In [5]:
targets={}
# Constants
#SPACE_TOKEN = '<space>'
#SPACE_INDEX = 0
#FIRST_INDEX = ord('a') - 96  # 0 is reserved to space


In [6]:
inputs = audio_to_mfcc('/home/saurabh/Documents/tf_orange/tf_orange/data/test.wav')

In [7]:
inputs.shape

(299, 15)

In [8]:
def decode_csv(line):
       parsed_line = tf.decode_csv(line, [[""],[""]])
       
    
       
       

       return parsed_line[0] , parsed_line[1]

In [9]:
# this function is required for CTC Loss
# for it's input , first convert transcrition / ground truth to number representation 

def sparse_tuple_from(sequences, dtype=np.int32):
    """Create a sparse representention of x.
    Args:
        sequences: a list of lists of type dtype where each element is a sequence
    Returns:
        A tuple with (indices, values, shape)
    """
    indices = []
    values = []

    for n, seq in enumerate(sequences):
        indices.extend(zip([n]*len(seq), range(len(seq))))
        values.extend(seq)

    indices = np.asarray(indices, dtype=np.int64)
    values = np.asarray(values, dtype=dtype)
    shape = np.asarray([len(sequences), np.asarray(indices).max(0)[1]+1], dtype=np.int64)

    return indices, values, shape

In [10]:

input_txt_file='/media/saurabh/New Volume2/hindi_char/sanskrit/data_backup/sanskrit_final.txt'
with open(input_txt_file, 'r', encoding='utf-16le') as myfile:
    data=myfile.read() #.replace('\n', '')
graphemes={}
from uniseg.graphemecluster import grapheme_clusters
graphemes_list=[]
for i in grapheme_clusters(data):
    graphemes_list.append(i)

graphemes_list=list(set(graphemes_list))
print(len(graphemes_list))
for i,j in enumerate(graphemes_list):
    #print(i)
    graphemes[j]=i






95


In [11]:
char_black_list = "!@#$?.,|:()1234567890"

def remove_blacklisted_chars(input):
    for char in char_black_list:
        input = input.replace(char,"")
    return input

In [12]:
def sentence_to_grapheme(sentence):
    input_sentence_list=[]
    for i in grapheme_clusters(sentence):
    #input_sentence_list.append(i)
     # return dictionary value of i
        
        #print(i)
        
        input_sentence_list.append(graphemes[i])
    return input_sentence_list
    

In [13]:
a = u"अधनस्य कुतो मित्रम्, अमित्रस्य कुतः"

sentence_to_grapheme(remove_blacklisted_chars(a))

[8,
 21,
 49,
 67,
 58,
 35,
 53,
 33,
 35,
 34,
 74,
 50,
 16,
 35,
 8,
 34,
 74,
 50,
 67,
 58,
 35,
 53,
 88]

In [14]:
graphemes_list

['रा',
 'री',
 'ष्',
 'दी',
 'थो',
 'स',
 'नु',
 'व',
 'अ',
 'आ',
 'र्',
 'न्',
 'उ',
 'रि',
 'धः',
 'ग',
 'म्',
 'दा',
 'सु',
 'टा',
 'टु',
 'ध',
 'वं',
 'नं',
 'त',
 'रु',
 ',',
 'ये',
 'पि',
 'सू',
 'घु',
 'थ',
 'या',
 'तो',
 'मि',
 ' ',
 'धै',
 'प',
 '\ufeff',
 'ध्',
 'घ',
 'वि',
 'रो',
 'द्',
 'नि',
 'दै',
 'ख',
 'जः',
 'नां',
 'न',
 'र',
 'का',
 'ना',
 'कु',
 'रे',
 'लो',
 'णे',
 'ब',
 'य',
 'हि',
 'लं',
 'व्',
 'पुः',
 'के',
 'ल',
 'ति',
 'माः',
 'स्',
 'यं',
 'क',
 'तु',
 'ह्',
 'ता',
 'द',
 'त्',
 '\xa0',
 'हा',
 'था',
 'ए',
 'सि',
 'च',
 'सा',
 'म',
 'क्',
 'श',
 'चे',
 'भ',
 'णां',
 'तः',
 'ण',
 'शी',
 'वे',
 'पु',
 'षु',
 'ष']

In [15]:
graphemes

{' ': 35,
 ',': 26,
 '\xa0': 75,
 'अ': 8,
 'आ': 9,
 'उ': 12,
 'ए': 78,
 'क': 69,
 'का': 51,
 'कु': 53,
 'के': 63,
 'क्': 83,
 'ख': 46,
 'ग': 15,
 'घ': 40,
 'घु': 30,
 'च': 80,
 'चे': 85,
 'जः': 47,
 'टा': 19,
 'टु': 20,
 'ण': 89,
 'णां': 87,
 'णे': 56,
 'त': 24,
 'तः': 88,
 'ता': 72,
 'ति': 65,
 'तु': 70,
 'तो': 33,
 'त्': 74,
 'थ': 31,
 'था': 77,
 'थो': 4,
 'द': 73,
 'दा': 17,
 'दी': 3,
 'दै': 45,
 'द्': 43,
 'ध': 21,
 'धः': 14,
 'धै': 36,
 'ध्': 39,
 'न': 49,
 'नं': 23,
 'ना': 52,
 'नां': 48,
 'नि': 44,
 'नु': 6,
 'न्': 11,
 'प': 37,
 'पि': 28,
 'पु': 92,
 'पुः': 62,
 'ब': 57,
 'भ': 86,
 'म': 82,
 'माः': 66,
 'मि': 34,
 'म्': 16,
 'य': 58,
 'यं': 68,
 'या': 32,
 'ये': 27,
 'र': 50,
 'रा': 0,
 'रि': 13,
 'री': 1,
 'रु': 25,
 'रे': 54,
 'रो': 42,
 'र्': 10,
 'ल': 64,
 'लं': 60,
 'लो': 55,
 'व': 7,
 'वं': 22,
 'वि': 41,
 'वे': 91,
 'व्': 61,
 'श': 84,
 'शी': 90,
 'ष': 94,
 'षु': 93,
 'ष्': 2,
 'स': 5,
 'सा': 81,
 'सि': 79,
 'सु': 18,
 'सू': 29,
 'स्': 67,
 'हा': 76,
 'हि': 59,
 'ह्': 71

In [16]:
# define a function, that pads audio so that audio frames = max frames

def pad(input):
   # print(input.shape[0])
     if input.shape[0] < timesteps:
        
        diff = timesteps - input.shape[0]
        
        # pad and return input
        return np.pad(input,((0,diff),(0,0)), mode="constant")
    
     elif input.shape[0] > timesteps:
        
        return input[:timesteps,:]

In [17]:
def _read_py_function(audio, label):
    audio = audio_to_mfcc(audio)
    
    if audio.shape[0] < timesteps:
        original_length=audio.shape[0]
       # print(original_length)
    
    elif audio.shape[0] >= timesteps:
        original_length=timesteps
        
    #audio=normalized(pad(audio) , axis=1 )
    audio=pad(audio)
   
    return audio ,label, original_length

In [18]:
dataset = tf.contrib.data.TextLineDataset("/home/saurabh/Documents/tf_orange/tf_orange/sanskrit.csv")
dataset=dataset.map(decode_csv)



dataset = dataset.map(
    lambda audio, label: tuple(tf.py_func(
        _read_py_function, [audio, label], [tf.double, label.dtype, tf.int64])))




dataset=dataset.batch(2)


Instructions for updating:
Use `tf.data.TextLineDataset`.


In [19]:

iterator = dataset.make_initializable_iterator()

item = iterator.get_next()

In [20]:
# Training Parameters
learning_rate = 0.01
training_steps = 100
#batch_size = 2
display_step = 200
num_features = 15
beta=0.01
logs_path = '/home/saurabh/Documents/tf_orange/tf_orange/hardik/logs'
model_path = '/home/saurabh/Documents/tf_orange/tf_orange/models/hardik/lite/model.ckpt'

# Network Parameters

timesteps = 300 # timesteps
num_hidden = 128 # hidden layer num of features

num_classes = len(graphemes_list) + 1



In [21]:
#prediction = tf.nn.softmax(logits)
inputs = tf.placeholder(tf.float32, [None, None, num_features])
targets = tf.sparse_placeholder(tf.int32)
seq_len = tf.placeholder(tf.int32, [None])
batch_size = 2

#batch_size = tf.placeholder ( tf.int32 )


In [22]:

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([  2 * num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]), name='bias')
}


In [23]:
'''
# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([  num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]), name='bias')
}
'''

"\n# Define weights\nweights = {\n    'out': tf.Variable(tf.random_normal([  num_hidden, num_classes]))\n}\nbiases = {\n    'out': tf.Variable(tf.random_normal([num_classes]), name='bias')\n}\n"

In [24]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell_fw = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    
    lstm_cell_bw = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    
    # Get lstm cell output
    outputs, _, _ = rnn.static_bidirectional_rnn(lstm_cell_fw, lstm_cell_bw, x,
    dtype=tf.float32)
    
    #convert output shape (timesteps * batch * classes ) to (batch*timesteps*classes)
    outputs=tf.transpose( outputs , [1, 0, 2])
    
    outputs=tf.reshape(outputs, [-1,2*num_hidden])
    
  
    res =  tf.matmul(outputs, weights['out']) + biases['out']
    
    res = tf.reshape(res, [batch_size,timesteps,num_classes])
    
    return res
   # return tf.nn.softmax(tf.matmul(outputs, weights['out']) + biases['out'])

In [25]:
'''def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
   # x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    
    

    
    # Get lstm cell output
   # outputs, _, _ = rnn.static_bidirectional_rnn(lstm_cell , x,
    #dtype=tf.float32)
    
    outputs, state = tf.nn.dynamic_rnn(cell=lstm_cell,
                                   inputs=x,
                                   sequence_length=seq_len,
                                   dtype=tf.float32)
    
    
    #convert output shape (timesteps * batch * hidden units ) to (batch*timesteps* hidden units)
    outputs=tf.transpose( outputs , [1, 0, 2])
    
    outputs=tf.reshape(outputs, [-1,num_hidden])
    
  
    res =  tf.matmul(outputs, weights['out']) + biases['out']
    
    res = tf.reshape(res, [batch_size,timesteps,num_classes])
    
    # new code 
    # state is of shape ( batch size ,  hidden_units)
    # need to adapt weights accordingly 
    
    
    return res
   # return tf.nn.softmax(tf.matmul(outputs, weights['out']) + biases['out']) '''

"def RNN(x, weights, biases):\n\n    # Prepare data shape to match `rnn` function requirements\n    # Current data input shape: (batch_size, timesteps, n_input)\n    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)\n\n    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)\n   # x = tf.unstack(x, timesteps, 1)\n\n    # Define a lstm cell with tensorflow\n    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)\n    \n    \n\n    \n    # Get lstm cell output\n   # outputs, _, _ = rnn.static_bidirectional_rnn(lstm_cell , x,\n    #dtype=tf.float32)\n    \n    outputs, state = tf.nn.dynamic_rnn(cell=lstm_cell,\n                                   inputs=x,\n                                   sequence_length=seq_len,\n                                   dtype=tf.float32)\n    \n    \n    #convert output shape (timesteps * batch * hidden units ) to (batch*timesteps* hidden units)\n    outputs=tf.transpose( outputs , [1, 0, 2])\n    \n   

In [26]:




logits = RNN(inputs, weights, biases)

loss =  tf.nn.ctc_loss ( targets, logits , seq_len , time_major = False)
cost = tf.reduce_mean(loss)

tv = tf.trainable_variables()
variables_names = [v.name for v in tf.trainable_variables()]

reg = tf.reduce_sum([ tf.nn.l2_loss(v) for v in tv if 'bias' not in v.name])

total_loss = tf.reduce_mean(cost + reg * beta)

optimizer = tf.train.MomentumOptimizer(learning_rate,
                                           0.9).minimize(total_loss)

gradient_optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

adam_optimizer = tf.train.AdamOptimizer(1e-4).minimize(cost)


# Option 2: tf.contrib.ctc.ctc_beam_search_decoder
    # (it's slower but you'll get better results)
decoder_input = tf.transpose(logits, [1, 0, 2])

decoded, log_prob = tf.nn.ctc_greedy_decoder(decoder_input, seq_len)

    

In [27]:
# Create a summary to monitor cost tensor
tf.summary.scalar("loss", cost)

# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

In [28]:
# 'Saver' op to save and restore all the variables
saver = tf.train.Saver()

In [29]:
init = tf.global_variables_initializer()

In [30]:
# need this for decoding word give its index key value 

def keys_of_value(dct, value):
    for k in dct:
        if isinstance(dct[k], list):
            if value in dct[k]:
                return k
        else:
            if value == dct[k]:
                return k

In [31]:
# testing above function 

print(keys_of_value(word_dictionary, 2419))

NameError: name 'word_dictionary' is not defined

In [32]:
# retrain from a checkpoint
# if training first time, ignore this block

with tf.Session() as sess:
    

    # Run the initializer
    sess.run(init)
        
    # Restore model weights from previously saved model
    load_path = saver.restore(sess, model_path +  '/2' )
    print("Model restored from file: %s" % load_path)
    
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

    # Training cycle
    for epoch in range(training_steps):
     #   avg_cost = 0.
           # Save model weights to disk
        save_path = saver.save(sess, model_path +  '/2')
        print("Model saved in file: %s" % save_path) 
        
        
        sess.run(iterator.initializer)
        train_cost=0
        
        while True:
            try:
                Next_element=sess.run(item)
             
            
                 
                print(Next_element[1])   
                
                batch_list = []
                
 
                for index,j in enumerate(Next_element[1]):
                    batch_list.insert(index, )
                    
                
                batch_targets = sparse_tuple_from(batch_list)
                
                print(Next_element[2])
                
                feed = {inputs: Next_element[0],
                        targets: batch_targets,
                        seq_len: Next_element[2],
                         }
	    

                batch_cost, _ , summary = sess.run([cost, adam_optimizer , merged_summary_op ], feed)
               # print(batch_cost)
                
                train_cost += batch_cost*batch_size
                
                
                # Write logs at every iteration
                summary_writer.add_summary(summary, epoch )
                
                 # Decoding
                d = sess.run(decoded[0], feed_dict=feed)
   
                dense_decoded = tf.sparse_tensor_to_dense(d, default_value=-1).eval(session=sess)
    
                for i, seq in enumerate(dense_decoded):

                    seq = [s for s in seq if s != -1]
       
                    #str_decoded = ''.join([keys_of_value(word_dictionary, x) for x in seq ])
                    #print(str_decoded)
            
              
            except tf.errors.OutOfRangeError:
              print(train_cost)
              
              
              break
                
        # Save model weights to disk at the end of all epochs, if we remove this last epoch doesnt gets saved
    save_path = saver.save(sess, model_path +  '/2')
    print("Model saved in file: %s" % save_path)        
    
    

INFO:tensorflow:Restoring parameters from /home/saurabh/Documents/tf_orange/tf_orange/models/hardik/lite/model.ckpt/2


InvalidArgumentError: Unsuccessful TensorSliceReader constructor: Failed to get matching files on /home/saurabh/Documents/tf_orange/tf_orange/models/hardik/lite/model.ckpt/2: Not found: /home/saurabh/Documents/tf_orange/tf_orange/models/hardik/lite/model.ckpt; No such file or directory
	 [[Node: save/RestoreV2_25 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save/Const_0_0, save/RestoreV2_25/tensor_names, save/RestoreV2_25/shape_and_slices)]]

Caused by op 'save/RestoreV2_25', defined at:
  File "/home/saurabh/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/saurabh/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/saurabh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/saurabh/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658

In [ ]:
# Start training
with tf.Session() as sess:
    sess.run(init)
    
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

    # Training cycle
    for epoch in range(training_steps):
     #   avg_cost = 0.
           # Save model weights to disk
        save_path = saver.save(sess, model_path)
        print("Model saved in file: %s" % save_path) 
        
        
        sess.run(iterator.initializer)
        train_cost=0
        
        while True:
            try:
                Next_element=sess.run(item)
             
            
                 
                #print(Next_element[1].decode("utf-8"))   
                
                #batch_list=np.zeros(2)
                
                batch_list = []
                
 
                for index,j in enumerate(Next_element[1]):
                    batch_list.insert(index, sentence_to_grapheme(j.decode("utf-8")))
                    
                
                batch_targets = sparse_tuple_from(batch_list)
                
                #batch_list =  sentence_to_char_array(Next_element[1][0].decode("utf-8"))
                    
                
                #batch_targets = sparse_tuple_from(batch_list)
                print(batch_list)
                
                print(Next_element[2])
                
                feed = {inputs: Next_element[0],
                        targets: batch_targets,
                        seq_len: Next_element[2],
                         }
	    

                batch_cost, _ , summary = sess.run([cost, gradient_optimizer , merged_summary_op ], feed)
               # print(batch_cost)
                
                train_cost += batch_cost*batch_size
                
                
                # Write logs at every iteration
                summary_writer.add_summary(summary, epoch )
                
                 # Decoding
                d = sess.run(decoded[0], feed_dict=feed)
   
                dense_decoded = tf.sparse_tensor_to_dense(d, default_value=-1).eval(session=sess)
    
                for i, seq in enumerate(dense_decoded):

                    seq = [s for s in seq if s != -1]
                
                    #print(seq)
       
                    str_decoded = ''.join([ graphemes_list[x]  for x in seq ])
                    print('decoded :' + str_decoded)
            
              
            except tf.errors.OutOfRangeError:
              print(train_cost)
              
              
              break
                
        # Save model weights to disk at the end of all epochs, if we remove this last epoch doesnt gets saved
    save_path = saver.save(sess, model_path)
    print("Model saved in file: %s" % save_path)         

               
        
            
                
                
             
                
                
            